In [2]:
begin
    using Pkg
    dev_folder = "../Examples/" # folder of the development environment
    pkg_folder = "../" # folder of the package
    Pkg.activate(dev_folder)
    Pkg.develop(path=pkg_folder)
end
# import Markdown; Base.showable(::MIME"text/markdown", ::Markdown.MD) = false # fix render problem for vscode notebook
Threads.nthreads() # check number of threads

  Activating project at `~/BindingAndCatalysis.jl/Examples`
   Resolving package versions...
     Project No packages added to or removed from `~/BindingAndCatalysis.jl/Examples/Project.toml`
    Manifest No packages added to or removed from `~/BindingAndCatalysis.jl/Examples/Manifest.toml`


12

In [ ]:
using BindingAndCatalysis # import the package
using CairoMakie # for plotting, we use Makie backend, could also be GLMakie or WGLMakie...

[ Info: Precompiling BindingAndCatalysis [b532a4b1-2c45-4c12-bcaf-8695372aa40c] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of t

# This is the minimal example for using BindingAndCatalysis package

---

## Initialize the model

For a binding network, one could define that by passing $N$ matrix and $L$ matrix, where N stands for stochiometry, with $N\log x = \log K$ , and $L$ is the conservation matrix, with $q=Lx$, one could pass either $L$ or $N$ to constract model, the other side will be calculated by $NL^\top=0$. 

(while worth to mention that, the automatedly calculated $N$ and $L$ may not be the bases you want, typically we recommand you pass $N$ and make atomic species at first)

Below it the example for basic monomer dimerization network

$$E+S\Longleftrightarrow C$$

with Conservation
$$tE= E+C $$
$$tS= S+C $$
And equilirium
$$ES=KC$$

In [4]:
model = let
    N = [1 1 -1]  # define stoichiometry matrix
    x_sym = [:E, :S, :C] # Optional: define species symbols
    q_sym = [:tE, :tS] # Optional: define total concentration symbols
    K_sym = [:K] # Optional: define binding constant symbols
    Bnc(N = N, x_sym=x_sym, q_sym=q_sym, K_sym=K_sym) # create Bnc model
end

----------Binding Network Summary:-------------
Number of species (n): 3
Number of conserved quantities (d): 2
Number of reactions (r): 1
L matrix: [1 0 1; 0 1 1]
N matrix: [1 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: No
-----------------------------------------------

In [5]:
model.L  # get L matrix of model

2×3 Matrix{Int64}:
 1  0  1
 0  1  1

In [6]:
show_conservation(model)# display conservation relations, q=Lx

2-element Vector{Symbolics.Equation}:
 tE ~ C + E
 tS ~ C + S

In [7]:
show_equilibrium(model,log_space=true) |> display # display equilibrium concentrations in log space
show_equilibrium(model,log_space=false) # display equilibrium concentrations in linear space

1-element Vector{Symbolics.Equation}:
 log10(K) ~ -log10(C) + log10(S) + log10(E)

1-element Vector{Symbolics.Equation}:
 K ~ (E*S) / C

In [8]:
logder_qK_x_sym(model) # get the symbolic log-derivative matrix of qK with respect to x
# equivalent to ∂logqK_∂logx_sym(model) 

3×3 Matrix{Symbolics.Num}:
 E / tE       0  C / tE
      0  S / tS  C / tS
      1       1      -1

In [9]:
# USE WITH CAUTION: simbolic invervse may be very slow for large models
logder_x_qK_sym(model) # get the symbolic log-derivative matrix of qK with respect to x
# equivalent to ∂logx_∂logqK_sym(model)

3×3 Matrix{Symbolics.Num}:
 (-C*tE - S*tE) / (-C*E - C*S - E*S)  …  (-C*S) / (-C*E - C*S - E*S)
         (C*tE) / (-C*E - C*S - E*S)     (-C*E) / (-C*E - C*S - E*S)
        (-S*tE) / (-C*E - C*S - E*S)      (E*S) / (-C*E - C*S - E*S)

In [10]:
x_sym(model) |> display # get species symbols
q_sym(model) |> display# get total concentration symbols
K_sym(model) |> display# get binding constant symbols
qK_sym(model) # get all symbols (q and K)

3-element Vector{Symbolics.Num}:
 E
 S
 C

2-element Vector{Symbolics.Num}:
 tE
 tS

1-element Vector{Symbolics.Num}:
 K

3-element Vector{Symbolics.Num}:
 tE
 tS
  K

In [11]:
summary(model) # display a summary of the model, currently we haven't find any regimes 

----------Binding Network Summary:-------------
Number of species (n): 3
Number of conserved quantities (d): 2
Number of reactions (r): 1
L matrix: [1 0 1; 0 1 1]
N matrix: [1 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: No
-----------------------------------------------


## Regimes/vertices:

---

Regimes or vertices is defined when every $q$ is dominated by a $x$, for example, for this binding network, one regime could be $tS\approx S$ and $tE\approx E$, which is equivalent to $S\gg C$, $E\gg C$, which gives conditions for $x$ space, which is inherently a polyhedron, $qK$ space condition is calculated my affinly mapping this polyhedron based on regime-specific dominant contition(rather than conservation) and original equilibrium. In summary We have:

- Dom condition: $q=\tilde Px$ or $\log q = P \log x + P_0 $ (quivalent to vector perm, which tells $q_i$ is dominanted by x_{perm_{i}})
- $f((q,K)\rightarrow x)$,($x$ expression): $\log x = H \log q,K + H_0$
- $f(x\rightarrow (q,K))$,($q,K$ expression): $\log q,K = M \log x + M_0$
- $x$ condition: $C_x \log x + C_{0,x} > 0$
- $q,K$ condition: $C_{qK} \log x + C_{0,qK} \geq 0$
- Volume: Fraction of space that meets $C_{qK} \log x \geq 0$ 


In [12]:
find_all_vertices!(model) # find all possible vertices

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 4 vertices found and 4 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.


4-element Vector{Vector{Int8}}:
 [1, 2]
 [1, 3]
 [3, 2]
 [3, 3]

In [13]:
get_vertices_perm_dict(model)# show the idx for all regimes' perm

Dict{Vector{Int8}, Int64} with 4 entries:
  [1, 3] => 2
  [3, 3] => 4
  [1, 2] => 1
  [3, 2] => 3

In [14]:
summary(model) # Now regime data is available

----------Binding Network Summary:-------------
Number of species (n): 3
Number of conserved quantities (d): 2
Number of reactions (r): 1
L matrix: [1 0 1; 0 1 1]
N matrix: [1 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: Yes
Number of regimes: 4
  - Invertible + Asymptotic: 3
  - Singular +  Asymptotic: 1
  - Invertible +  Non-Asymptotic: 0
  - Singular +  Non-Asymptotic: 0
-----------------------------------------------


The regime is classified as below:

1. Invertible vs singular
- Invertible: qK condition polyheron is full dimension; M is invertible
- Singular: qK condition polyhedron is not full dim(live in hyperplane); M is singular

2. Asymptotic vs Non-asymptotic
- Asymptotic: qK condition polyheron's recession cone (take all $C_0$ equals $0$) have same dimension as the initial polyhedron
- Non-Asymptotic: qK condition polyheron's recession cone have smaller dimension compared to initial polyhedron.

In [15]:
# To locate a regime, one  can model+ regime_idx/regime_perm
V1 = get_vertex(model,[1,2]) # fetch Vertex according to perm,
V2 = get_vertex(model,1) # fetch Vertex according to idx
V1 === V2

true

In [16]:
# when you have "Vertex", you can directly pass it as parameter without model
C₁, C0₁, nullity₁ = get_C_C0_nullity(V1)
C₂, C0₂, nullity₂ = get_C_C0_nullity(model,1)
C₃, C0₃, nullity_3 = get_C_C0_nullity(model,[1,2])
@assert C₁ == C₂ == C₃
@assert C0₁ == C0₂ == C0₃
@assert nullity₁ == nullity₂ == nullity_3

In [17]:
show_condition_x(V1)

2-element Vector{Symbolics.Num}:
 (-log10(C) + log10(E)) > 0
 (-log10(C) + log10(S)) > 0

In [18]:
show_condition(model,[1,2];log_space=true) |> display
show_condition_qK(model,1;log_space=false) # show_condition is eqivalent to show_condition_qK

2-element Vector{Symbolics.Num}:
 (-log10(tS) + log10(K)) > 0
 (-log10(tE) + log10(K)) > 0

2-element Vector{Symbolics.Num}:
 K > tS
 K > tE

In [19]:
get_perm(V1)

2-element Vector{Int8}:
 1
 2

In [20]:
get_idx(model,[1,2])

1

In [21]:
get_nullity(model,1)

0

In [22]:
is_singular(V1)

false

In [23]:
is_asymptotic(model,1)

true

In [24]:
# Get properties of regimes
get_P_P0(model,[1,2]) # get P and P0
get_P(model,1)
get_P0(V1)

get_H_H0(model,1) # get H and H0
get_H(V1)
get_H0(model,[1,2])

# x space conditon 
get_C_C0_x(model,1)
get_C_x(model,[1,2])
get_C0_x(V1)

# qK space condition
get_C_C0_nullity(model,[1,2]) # equal to get_C_C0_nullity_qK
get_C_C0(model,1) # equal to get_C_C0
get_C0(V1) # equal to get_C0
get_C(V1) # equal to get_C

2×3 SparseArrays.SparseMatrixCSC{Float64, Int64} with 4 stored entries:
   ⋅   -1.0  1.0
 -1.0    ⋅   1.0

In [25]:
get_volume(model,1) # get volume of regime 1

[ Info: Number of polyhedra to calc volume: 1
[ Info: All regimes converged after 400000 samples.


(0.332994103865682, 0.0014604909038193755)

In [26]:
get_one_inner_point(model,2) # get one point that is sure to within specific regime

3-element Vector{Float64}:
  1.4149616889438872
 -1.552777003018668
  0.19154309589857446

In [27]:
poly = get_polyhedron(model,[1,2]) # get polyhedron denotes qK condition, backend using Polyhedra.jl and CDDLib.jl

# get_polyhedron(C,C0,nullity) is also valid

Polyhedron CDDLib.Polyhedron{Float64}:
2-element iterator of Polyhedra.HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([-0.0, 1.0, -1.0], 0.0)
 HalfSpace([1.0, -0.0, -1.0], 0.0)

In [28]:
# For polyheron we could also directly get C, C0 and nullity 

get_C_C0_nullity(poly) # express polyhedra as Cx+C0 >=0
get_C_C0(poly)
get_C0(poly)
get_C(poly)

get_nullity(poly)

0

In [29]:
get_one_inner_point(poly)

3-element Vector{Float64}:
 -2.5992162460096657
 -3.8567914095233693
  0.17782598368520836

In [30]:
calc_volume(poly)

[ Info: Number of polyhedra to calc volume: 1
[ Info: All regimes converged after 400000 samples.


(0.33318910199298885, 0.0014607049040022996)

In [31]:
summary(V1)

idx=1,perm=Int8[1, 2], asymptotic=true, nullity=0
volume=0.332994103865682 +- 0.0014604909038193755
Dominante condition


tE ~ E

tS ~ S

x expression
condition:


K > tS

K > tE

In [32]:
get_vertices_volume!(model) # compute volumes for all regimes

[ Info: Number of polyhedra to calc volume: 2
[ Info: All regimes converged after 400000 samples.


4-element Vector{Tuple{Float64, Float64}}:
 (0.332994103865682, 0.0014604909038193755)
 (0.33310160283329987, 0.0014606089130801423)
 (0.3336690973832825, 0.0014612304754686853)
 (0.0, 0.0)

### get regimes with specific properties

In [33]:
get_vertices(model; singular=true, return_idx=true)

1-element Vector{Int64}:
 4

## Check regime feasibility with additional constrains

In [34]:
 # for example, we need tE>=2*tS, 
#  which is equivalent to (logtE - log tS)+ (- log 2) >= 0
C_add = [1 -1 0] # add extra constraint C_add * x + C0_add >=0
C0_add = [-log10(2)]

# check if regime 1 is feasible with the extra constraint
check_feasibility_with_constraint(model,4; C= C_add,C0= C0_add)

[ Info: The dimension of the intersected polyhedra is 0


false

In [35]:
# get all feasible vertices with the extra constraint
feasible_vertieces_with_constraint(model; C= C_add,C0= C0_add, return_idx=true)

[ Info: The dimension of the intersected polyhedra is 3
[ Info: The dimension of the intersected polyhedra is 3
[ Info: The dimension of the intersected polyhedra is 0
[ Info: The dimension of the intersected polyhedra is 0


2-element Vector{Int64}:
 1
 2

## regime graph part

## Numerical part

Mapping directly between x space and qK space

In [36]:
logqK_vec = randomize(model,4;log_lower=-6, log_upper=6) # generate a vector of qK

4-element Vector{Vector{Float64}}:
 [5.212180865121374, -5.21968224537893, -4.816187154687737]
 [-0.013345554014349226, 4.28399217238203, -4.563464852588316]
 [-1.3005817282421912, -2.432879734734636, -0.03343910790806781]
 [3.949980858083837, 2.1857684790057714, -2.0744351757658674]

In [37]:
logx_vec = logqK_vec .|> qK-> qK2x(model,qK; input_logspace=true, output_logspace=true)# map qK vector to x space

4-element Vector{Vector{Float64}}:
 [5.212180865105307, -15.248050265212656, -5.219682245419613]
 [-8.860780678925064, 4.283970271705307, -0.013345554631440189]
 [-1.3022213481353246, -2.4556601724665423, -3.7244424126937994]
 [3.9424414760208664, -3.831108590522298, 2.1857680612644357]

In [38]:
logqK_vec2 = logx_vec .|> x-> x2qK(model,x; input_logspace=true, output_logspace=true) # map x vector back to qK space

4-element Vector{Vector{Float64}}:
 [5.212180865121374, -5.21968224537893, -4.816187154687735]
 [-0.013345554014349234, 4.28399217238203, -4.563464852588316]
 [-1.3005817282421914, -2.432879734734636, -0.03343910790806737]
 [3.949980858083837, 2.185768479005771, -2.0744351757658674]

In [39]:
logqK_vec2.-logqK_vec # check the difference, should be close to zero vector

4-element Vector{Vector{Float64}}:
 [0.0, 0.0, 1.7763568394002505e-15]
 [-8.673617379884035e-18, 0.0, 0.0]
 [-2.220446049250313e-16, 0.0, 4.440892098500626e-16]
 [0.0, -4.440892098500626e-16, 0.0]

Assign regime

In [40]:
logqK_vec .|> qK -> assign_vertex(model,qK; input_logspace=true, asymptotic_only=false, return_idx=true)

4-element Vector{Int64}:
 2
 3
 1
 2

In [41]:
logx_rgm = logqK_vec .|> qK -> qK2x(model,qK; input_logspace=true, output_logspace=true, use_vtx=true)

4-element Vector{Vector{Float64}}:
 [5.212180865121374, -15.248050265188041, -5.21968224537893]
 [-8.860802578984696, 4.28399217238203, -0.013345554014349226]
 [-1.3005817282421912, -2.432879734734636, -3.7000223550687594]
 [3.949980858083837, -3.838647554843933, 2.1857684790057714]

In [42]:
logx_rgm .- logx_vec # check the difference, should likely be close

4-element Vector{Vector{Float64}}:
 [1.6066259433955565e-11, 2.461497672356927e-11, 4.0683012514364236e-11]
 [-2.1900059632429247e-5, 2.1900676722808043e-5, 6.170909633979038e-10]
 [0.0016396198931334371, 0.0227804377319063, 0.02442005762503996]
 [0.007539382062970557, -0.007538964321634811, 4.1774133574534744e-7]

Draw simple plot to show the difference

In [ ]:
logK=1
logE_rg = (-6,6)
logS_rg = (-6,6)

total_points = 1e6 # points each dimension
#  minE, minS;          minE,maxS;
#
#  maxE; minS;          maxE,maxS;
pts_each_dim =round(Int, total_points^(1/2))

logx_mtx = let 
    data = data = Matrix{Vector{Float64}}(undef, pts_each_dim,pts_each_dim)
    
    # fill the first axis
    startqK = [logS_rg[1], logE_rg[1], logK]
    endqK = [logS_rg[2], logE_rg[1],logK]
    data[:,1] .= x_traj_with_qK_change(model,startqK,endqK; 
        input_logspace=true, output_logspace=true,npoints=pts_each_dim,ensure_manifold=true)[2]

    # fill the second axis
    start_logS = range(logS_rg..., length=pts_each_dim)
    Threads.@threads for i in 1:points
        startqK = [start_logS[i], logE_rg[1], logK]
        endqK = [start_logS[i], logE_rg[2], logK]
        start_logx = data[i,1]
        data[i,1:end] .= x_traj_with_qK_change(model, startqK, endqK; 
            startlogx = start_logx, 
            input_logspace=true, output_logspace=true,npoints=pts_each_dim,ensure_manifold=true)[2]
    end
    data
end


